<h4>Transfer Learning</h4>

<h5>Imports</h5>

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from typing import Callable
from sklearn import svm
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from tensorflow.python.profiler import profiler_v2 as profiler
import visualkeras
from PIL import ImageFont
import time
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, precision_recall_curve
import matplotlib.pyplot as plt
import math
import pickle

2023-06-10 22:10:04.603245: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-10 22:10:04.740467: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 22:10:05.685786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#loda all payloads from txt files in dataset folder (where each payload is one line)
def load_payloads(dataset_path: str) -> list:
    payloads = []
    for filename in os.listdir(dataset_path):
        with open(dataset_path + filename, 'r') as f:
            payloads.extend(f.readlines())
    return payloads

In [14]:
def parse_http_requests(file_path):
    requests = []
    current_request = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line == '':
                if current_request:
                    if (current_request['method'] == 'POST' or current_request['method'] == 'PUT') and 'body' not in current_request.keys():
                        continue    
                    requests.append(current_request)
                    current_request = {}
            elif line.startswith('GET') or line.startswith('POST') or line.startswith('PUT'):
                method, uri, _ = line.split(' ')
                current_request['method'] = method
                current_request['uri'] = uri
            else:
                if ': ' not in line:
                    current_request['body'] = line
                else:
                    header_name, header_value = line.split(': ', 1)
                    if 'headers' not in current_request:
                        current_request['headers'] = {}
                    current_request['headers'][header_name] = header_value
    if current_request:
        requests.append(current_request)
    return requests


file_paths = ['./dataset/cisc/normalTrafficTraining.txt', './dataset/cisc/anomalousTrafficTest.txt', './dataset/cisc/normalTrafficTest.txt']
cisc_train = parse_http_requests(file_paths[0])
cisc_test_malicious = parse_http_requests(file_paths[1])
cisc_test_normal = parse_http_requests(file_paths[2])

def parse_requests(requests):
    data=[]
    for request in requests:
        if request['method'] == 'POST':
            data.append('POST'+' '+request['uri']+' '+request['body']+' '+json.dumps(request['headers']))
        else:
            data.append('GET'+' '+request['uri']+' '+json.dumps(request['headers']))
    return data

payloads = shuffle(load_payloads('dataset/payloads/'),random_state=0)[:2500]   

train_examples = parse_requests(cisc_train)[:16000]
normal = parse_requests(cisc_test_normal)
malicious = parse_requests(cisc_test_malicious)
test_examples = normal+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal)
test_labels.extend([1] * len(malicious))

payload_iterator={}
payload_iterator[0] = 0
def get_next_payload():
    payload_iterator[0] = (payload_iterator[0] + 1) % len(payloads)
    return payloads[payload_iterator[0]]

#create synthetic malicious data by adding payloads to normal data (PAYLOAD SHOULD BE injected at a space character )
tmp = train_examples.copy()
np.random.shuffle(payloads)
synthetic_malicious = []
for i in range(len(tmp)):
    payload = get_next_payload()
    tmp[i] = tmp[i].split(' ') 
    tmp[i].insert(np.random.randint(0,len(tmp[i])), payload)
    tmp[i] = ' '.join(tmp[i])
    synthetic_malicious.append(tmp[i])

#train_examples.extend(synthetic_malicious[:8000])
#train_labels.extend([1] * len(synthetic_malicious[:8000]))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))

#shuflee datasets with given radnom seed
dataset_train = dataset_train.shuffle(400000, seed=42, reshuffle_each_iteration=False)
dataset_test = dataset_test.shuffle(400000, seed=42, reshuffle_each_iteration=False)

def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())


b'a a: /  / a:n / x / a / a.a ? a = a a  { "a-a": "a / n.n  ( a; a / n.n; a )  a / n.n.n  ( a a ) ", "a": "a-a", "a-a": "a-a", "a": "a / a,a / a,a / a a,a / a;a = n.n,a / a;a = n.n,a / a,* / *;a = n.n", "a-a": "a-a, a-a, a, a", "a-a": "a-n, a-n;a = n.n, *;a = n.n", "a-a": "a", "a": "a:n", "a": "a = x", "a": "a" } ' 0
b'a   a : / / a : n / x / a / a . a ? a = a   a   { " a - a " :   " a / n . n   ( a ;   a / n . n ;   a )   a / n . n . n   ( a   a ) " ,   " a " :   " a - a " ,   " a - a " :   " a - a " ,   " a " :   " a / a , a / a , a / a   a , a / a ; a = n . n , a / a ; a = n . n , a / a , * / * ; a = n . n " ,   " a - a " :   " a - a ,   a - a ,   a ,   a " ,   " a - a " :   " a - n ,   a - n ; a = n . n ,   * ; a = n . n " ,   " a - a " :   " a " ,   " a " :   " a : n " ,   " a " :   " a = x " ,   " a " :   " a " } ' 0


2023-06-10 22:44:24.469247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [16000]
	 [[{{node Placeholder/_1}}]]
2023-06-10 22:44:24.506514: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [16000]
	 [[{{node Placeholder/_1}}]]


In [15]:
len(train_examples)

16000

<h4>Save and load vectorization from disk</h4>

In [16]:

from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-06-10 22:44:24.593465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-06-10 22:44:24.698695: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>Load and retrain all models</h4>

In [17]:
#load lstm model
lstm_model = tf.keras.models.load_model('lstm/my_model')
lstm_model_sub = tf.keras.models.load_model('lstm_sub/my_model')

#load transformer model
transformer_model = tf.keras.models.load_model('trans/my_model')
transformer_model_sub = tf.keras.models.load_model('trans_sub/my_model')

#load cnn model
cnn_model = tf.keras.models.load_model('cnn/my_model') 
cnn_model_sub = tf.keras.models.load_model('cnn_sub/my_model') 

#load autoencoder model
autoencoder_model = tf.keras.models.load_model('autoencoder/my_model')
autoencoder_model_sub = tf.keras.models.load_model('autoencoder_sub/my_model')

#create new svm model   
svm_model = svm.OneClassSVM(gamma='auto')
svm_model_sub = svm.OneClassSVM(gamma='auto')


2023-06-10 22:44:29.530408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:29.530574: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:44:29.530628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:44:29.990322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:44:29.991391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:29.991448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:44:30.985208: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:44:30.986298: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:30.986356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:44:31.217377: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:31.217554: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:44:31.217615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:44:31.762685: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:44:31.763777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:31.763834: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:44:32.672530: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:44:32.674919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:32.675001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:44:32.891366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:32.891505: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:44:32.891571: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:44:33.429986: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:33.430123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:44:33.430171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:44:33.948341: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:44:33.948497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:44:33.948550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
#prepare train data
def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label

def autoencoder_processing(text, label):
    return  vectorization(text), vectorization(text)

def autoencoder_sub_processing(text, label):
    return  vectorization_sub(text), vectorization_sub(text)

auto_train_auto_sub = train_sub.map(autoencoder_sub_processing)
auto_train_auto = train.map(autoencoder_processing)

train_auto_sub = train_sub.map(sub_processing)
train_auto = train.map(processing)

# convert tf Dataset to numpy array
svm_train_auto = np.array(list(train_auto.as_numpy_iterator()))
svm_train_auto_sub = np.array(list(train_auto_sub.as_numpy_iterator()))

#extract the text and label from the numpy array
train_text = svm_train_auto[:,0]
train_label = svm_train_auto[:,1]
train_text_sub = svm_train_auto_sub[:,0]
train_label_sub = svm_train_auto_sub[:,1]

train_auto = train_auto.batch(32)
train_auto_sub = train_auto_sub.batch(32)

auto_train_auto_sub = auto_train_auto_sub.batch(32)
auto_train_auto = auto_train_auto.batch(32)

2023-06-10 22:44:39.218745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]
/tmp/ipykernel_7646/3845410109.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  svm_train_auto = np.array(list(train_auto.as_numpy_iterator()))
2023-06-10 22:44:43.036383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 

In [19]:
#retrain models and save
#cnn_model.fit(train_auto, epochs=10)
#cnn_model.save('cisc/cnn/my_model')
#cnn_model_sub.fit(train_auto_sub, epochs=10)
#cnn_model_sub.save('cisc/cnn_sub/my_model')

#transformer_model.fit(train_auto, epochs=10)
#transformer_model.save('cisc/transformer/my_model')
#transformer_model_sub.fit(train_auto_sub, epochs=10)
#transformer_model_sub.save('cisc/transformer_sub/my_model')

#lstm_model.fit(train_auto, epochs=10)
#lstm_model.save('cisc/lstm/my_model')
#lstm_model_sub.fit(train_auto_sub, epochs=10)
#lstm_model_sub.save('cisc/lstm_sub/my_model')

autoencoder_model.fit(auto_train_auto, epochs=10)
autoencoder_model.save('cisc/autoencoder/my_model')
autoencoder_model_sub.fit(auto_train_auto_sub, epochs=10)
autoencoder_model_sub.save('cisc/autoencoder_sub/my_model')

svm_model = svm_model.fit(list(train_text))
filename = './cisc/svm/svm_model.sav'
pickle.dump(svm_model, open(filename, 'wb'))
svm_model_sub = svm_model_sub.fit(list(train_text_sub))
filename = './cisc/svm_sub/svm_model.sav'
pickle.dump(svm_model_sub, open(filename, 'wb'))

2023-06-10 22:44:46.635077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]


Epoch 1/10
500/500 [==============================] - 5s 8ms/step - loss: 7.1808
Epoch 2/10
500/500 [==============================] - 5s 9ms/step - loss: 6.0217
Epoch 3/10
500/500 [==============================] - 4s 9ms/step - loss: 5.8097
Epoch 4/10
500/500 [==============================] - 4s 9ms/step - loss: 5.6620
Epoch 5/10
500/500 [==============================] - 4s 9ms/step - loss: 5.5706
Epoch 6/10
500/500 [==============================] - 4s 8ms/step - loss: 5.4901
Epoch 7/10
500/500 [==============================] - 4s 8ms/step - loss: 5.4484
Epoch 8/10
500/500 [==============================] - 4s 8ms/step - loss: 5.4053
Epoch 9/10
500/500 [==============================] - 4s 8ms/step - loss: 5.3581
Epoch 10/10
500/500 [==============================] - 5s 10ms/step - loss: 5.3384
INFO:tensorflow:Assets written to: cisc/autoencoder/my_model/assets


INFO:tensorflow:Assets written to: cisc/autoencoder/my_model/assets


Epoch 1/10


2023-06-10 22:45:32.120963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [16000]
	 [[{{node Placeholder/_1}}]]


500/500 [==============================] - 5s 7ms/step - loss: 11635.5850
Epoch 2/10
500/500 [==============================] - 4s 7ms/step - loss: 10885.0059
Epoch 3/10
500/500 [==============================] - 4s 7ms/step - loss: 10705.0449
Epoch 4/10
500/500 [==============================] - 4s 7ms/step - loss: 10615.0811
Epoch 5/10
500/500 [==============================] - 4s 7ms/step - loss: 10538.1465
Epoch 6/10
500/500 [==============================] - 4s 7ms/step - loss: 10526.1406
Epoch 7/10
500/500 [==============================] - 4s 7ms/step - loss: 10510.2217
Epoch 8/10
500/500 [==============================] - 4s 7ms/step - loss: 10517.0244
Epoch 9/10
500/500 [==============================] - 4s 7ms/step - loss: 10499.6436
Epoch 10/10
500/500 [==============================] - 4s 8ms/step - loss: 10505.8916
INFO:tensorflow:Assets written to: cisc/autoencoder_sub/my_model/assets


INFO:tensorflow:Assets written to: cisc/autoencoder_sub/my_model/assets


In [20]:
#load retrained models
cnn_model= tf.keras.models.load_model('cisc/cnn/my_model')
cnn_model_sub= tf.keras.models.load_model('cisc/cnn_sub/my_model')
transformer_model= tf.keras.models.load_model('cisc/transformer/my_model')
transformer_model_sub= tf.keras.models.load_model('cisc/transformer_sub/my_model')
lstm_model= tf.keras.models.load_model('cisc/lstm/my_model')
lstm_model_sub= tf.keras.models.load_model('cisc/lstm_sub/my_model')
autoencoder_model= tf.keras.models.load_model('cisc/autoencoder/my_model')
autoencoder_model_sub= tf.keras.models.load_model('cisc/autoencoder_sub/my_model')
filename = './cisc/svm/svm_model.sav'
svm_model=pickle.load( open(filename, 'rb'))
filename = './cisc/svm_sub/svm_model.sav'
svm_model_sub=pickle.load(open(filename, 'rb'))

2023-06-10 22:46:55.247858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:46:55.248904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:55.248963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:46:55.473328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:46:55.474402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:55.474462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:46:56.725452: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:56.725589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:46:56.725647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:46:57.335180: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:57.335347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:46:57.335399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:46:58.043566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 22:46:58.045014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 22:46:58.046040: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:46:58.589093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:46:58.590091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:58.590146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:46:59.038546: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:46:59.039786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:59.039877: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-06-10 22:46:59.361781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:59.361947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-10 22:46:59.361998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-06-10 22:46:59.943140: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_reversev2_grad_reversev2_reversev2_axis' with dtype int32 and shape [1]
	 [[{{node gradients_reversev2_grad_reversev2_reversev2_axis}}]]
2023-06-10 22:46:59.944270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-10 22:46:59.944333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

In [21]:
#prepare test data
#testdataset for normal_models:
def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label
test_auto_sub = test_sub.map(sub_processing)

test_auto = test.map(processing)



#tesdataset for svm
test_auto_svm = np.array(list(test_auto.as_numpy_iterator()))
test_auto_sub_svm = np.array(list(test_auto_sub.as_numpy_iterator()))
test_text = test_auto_svm[:,0]
test_label = test_auto_svm[:,1]
test_text_sub = test_auto_sub_svm[:,0]
test_label_sub = test_auto_sub_svm[:,1]


test_auto_sub = test_auto_sub.batch(32)
test_auto = test_auto.batch(32)
def get_data(x, y):
  return x

# Use the map() function to extract the labels
dataset_tmp = test_auto.map(get_data)
dataset_tmp_sub = test_auto_sub.map(get_data)

2023-06-10 22:47:02.012200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]
/tmp/ipykernel_7646/1756571173.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_auto_svm = np.array(list(test_auto.as_numpy_iterator()))
2023-06-10 22:47:16.125906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shap

In [22]:
#run and time predictions
def predict_and_time(model,model_sub,name):
    print(f"Running predictions for {name}")
    reconstructions = model.predict(test_auto, verbose=0)
    reconstructions_sub = model_sub.predict(test_auto_sub, verbose=0)
    return reconstructions, reconstructions_sub
def predict_and_time_svm(model,model_sub):
    print(f"Running predictions for svm")
    reconstructions = model.predict(list(test_text))
    reconstructions_sub = model_sub.predict(list(test_text_sub)) 
    return reconstructions, reconstructions_sub    

reconstructions_cnn, reconstructions_sub_cnn = predict_and_time(cnn_model,cnn_model_sub,"cnn")
reconstructions_trans, reconstructions_sub_trans = predict_and_time(transformer_model,transformer_model_sub,"transformer")
reconstructions_lstm, reconstructions_sub_lstm = predict_and_time(lstm_model,lstm_model_sub,"lstm")
reconstructions_auto, reconstructions_sub_auto = predict_and_time(autoencoder_model,autoencoder_model_sub,"autoencoder")
reconstructions_svm, reconstructions_sub_svm = predict_and_time_svm(svm_model,svm_model_sub)


2023-06-10 22:47:29.212820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [61065]
	 [[{{node Placeholder/_1}}]]


Running predictions for cnn


2023-06-10 22:47:43.655766: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]


Running predictions for transformer


2023-06-10 22:47:57.742521: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'x' with dtype float and shape [?,?]
	 [[{{node x}}]]
2023-06-10 22:48:12.678486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'x' with dtype float and shape [?,?]
	 [[{{node x}}]]


Running predictions for lstm


2023-06-10 22:48:28.033183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 22:48:28.034795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 22:48:28.035718: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Running predictions for autoencoder
Running predictions for svm


In [23]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def calc_and_print_metrics(y_test,reconsturctions,name, mse_ratio=0.5,pred_treshold=0.5):
    
    if(name.find("auto") != -1):
        if(name.find("sub") != -1):
            tmp_sub = np.concatenate(list(dataset_tmp_sub.as_numpy_iterator()))
            mse_ = tf.keras.losses.mean_squared_error(tmp_sub, reconsturctions)
            mse = [0.01*x for x in mse_]
        else:  
            tmp = np.concatenate(list(dataset_tmp.as_numpy_iterator()))
            mse = tf.keras.losses.mean_squared_error(tmp, reconsturctions)
        avg =  sum(mse) / len(mse)
        print(avg)
        reconsturctions=list(map(lambda x: sigmoid(x.numpy()-mse_ratio*avg.numpy()), list(mse)))
        y_pred = list(map(lambda x: 0 if sigmoid(x.numpy()-mse_ratio*avg.numpy())<pred_treshold else 1, list(mse)))
    elif(name.find("svm") != -1):
        y_pred = list(map(lambda x: 1 if x==-1 else 0, list(reconsturctions)))
    else:
        y_pred = list(map(lambda x: 0 if x[0]<pred_treshold else 1, list(reconsturctions)))
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    
    print(f"Metrics for model {name}")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("MCC:", mcc)

In [24]:
actual = test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))
calc_and_print_metrics(actual,reconstructions_cnn,"cnn",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_cnn,"cnn_sub",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_trans,"trans",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_trans,"trans_sub",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_lstm,"lstm",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_lstm,"lstm_sub",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_auto,"auto",0.3)
calc_and_print_metrics(actual,reconstructions_sub_auto,"auto_sub", 0.3)
calc_and_print_metrics(actual,reconstructions_svm,"svm",pred_treshold=0.1)
calc_and_print_metrics(actual,reconstructions_sub_svm,"svm_sub",pred_treshold=0.1)

2023-06-10 22:51:45.561365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [61065]
	 [[{{node Placeholder/_1}}]]


Metrics for model cnn
Accuracy: 0.6916400556783755
Precision: 0.9291121816930489
Recall: 0.26929982046678635
F1 Score: 0.4175688215279926
MCC: 0.38744188635585813
Metrics for model cnn_sub
Accuracy: 0.7146319495619422
Precision: 0.9994769190532234
Recall: 0.3049271893077997
F1 Score: 0.46729029102470043
MCC: 0.45303694391563515
Metrics for model trans
Accuracy: 0.7645132236141816
Precision: 0.9328364255043344
Recall: 0.45936564931178936
F1 Score: 0.6155902480752781
MCC: 0.5344844756525858
Metrics for model trans_sub
Accuracy: 0.6510275935478589
Precision: 0.99867197875166
Recall: 0.1500099740674247
F1 Score: 0.2608394033992369
MCC: 0.3065089466059254
Metrics for model lstm
Accuracy: 0.7318758699746172
Precision: 0.9104646769928221
Recall: 0.3846000398962697
F1 Score: 0.5407679578156116
MCC: 0.46551944044173227
Metrics for model lstm_sub
Accuracy: 0.6805207565708671
Precision: 1.0
Recall: 0.22166367444643925
F1 Score: 0.36288821397080434
MCC: 0.3791550219765141


2023-06-10 22:52:00.793707: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]


tf.Tensor(8.494152, shape=(), dtype=float32)
Metrics for model auto
Accuracy: 0.41046425939572584
Precision: 0.41046425939572584
Recall: 1.0
F1 Score: 0.5820271682340648
MCC: 0.0


2023-06-10 22:53:11.921862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]


tf.Tensor(153.63766, shape=(), dtype=float32)
Metrics for model auto_sub
Accuracy: 0.6786538933922869
Precision: 0.619625428646795
Recall: 0.5622980251346499
F1 Score: 0.5895714375352953
MCC: 0.32759235000771075
Metrics for model svm
Accuracy: 0.47768770981740766
Precision: 0.4381004168932391
Recall: 0.964292838619589
F1 Score: 0.6024802143702873
MCC: 0.17186779409787792
Metrics for model svm_sub
Accuracy: 0.7073610087611562
Precision: 0.591784666411532
Recall: 0.9253939756632755
F1 Score: 0.721910986616869
MCC: 0.49345196724246543


In [25]:
print("EEESS")

EEESS
